In [1]:
import os

In [2]:
# Change the current working directory to the project root folder
# This ensures the code works with relative paths and avoids FileNotFoundError
os.chdir("../")
%pwd

'd:\\Projects\\Apple-Disease-Classification-Project'

In [3]:
from dataclasses import dataclass
from pathlib import Path

# Data class for storing model preparation configuration
@dataclass(frozen=True)
class PrepareModelConfig:
    root_dir: Path
    built_model_path: Path    
    params_image_size: list
    params_n_classes: int
    params_learning_rate: float
    params_rho: float
    params_epsilon: float


In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
from src.cnnClassifier.logging import logger

In [5]:
# Configuration manager class for handling configuration setup and retrieval
class ConfigurationManager:
    
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        """
        Initialize ConfigurationManager by loading configuration settings from YAML files
        and creating necessary directories for artifacts.
        
        Args:
            config_filepath (str): Path to the configuration file.
            params_filepath (str): Path to the parameters file.
        """
        # Load configuration settings from YAML files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # Create necessary directories for artifacts
        create_directories([self.config.artifacts_root])

    def get_prepare_model_config(self) -> PrepareModelConfig:
        """
        Retrieve the configuration for preparing the model.

        Returns:
            PrepareModelConfig: An object containing the configuration settings for preparing the model.
        """
        # Extract the model preparation configuration from the loaded config
        config = self.config.model_preparation

        # Create directories specified in the model configuration
        create_directories([config.root_dir])

        # Instantiate and return the PrepareModelConfig object with the necessary settings
        prepare_model_config = PrepareModelConfig(
            root_dir=Path(config.root_dir),
            built_model_path=Path(config.built_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_n_classes=self.params.N_CLASSES,
            params_learning_rate=self.params.LEARNING_RATE,
            params_rho= self.params.RHO,
            params_epsilon= self.params.EPSILON,
        )

        return prepare_model_config

In [6]:
import tensorflow as tf
from keras import models, layers
from keras.optimizers import RMSprop 

In [7]:
# Class for preparing and building the model
class PrepareModel:
    """
    PrepareModel class is responsible for creating a custom model,
    compiling it, and saving it to specified paths.
    """

 

    def __init__(self, config: PrepareModelConfig):
        """
        Initialize the PrepareModel with the given configuration.

        Args:
            config (PrepareModelConfig): Configuration object for preparing the model.
        """
        self.config = config

    
    @staticmethod
    def prepare_model(input_shape, n_classes, learning_rate, rho, epsilon):
        """
        Prepare the model by defining its architecture and compiling it.

        Args:
            input_shape (tuple): Shape of the input images, derived from params_image_size in the config.
            n_classes (int): Number of output classes.
            learning_rate (float): Learning rate for the optimizer.
            rho (float): Decay rate for RMSprop optimizer.
            epsilon (float): Small value to prevent division by zero in RMSprop optimizer.

        Returns:
            tf.keras.Model: The compiled model.
        """

        # Define the model architecture
        model = models.Sequential([
                layers.InputLayer(input_shape=input_shape),
                layers.Conv2D(filters = 128, kernel_size = (3,3), kernel_regularizer=tf.keras.regularizers.l2(0.01), input_shape = input_shape),
                layers.BatchNormalization(),
                layers.Activation('relu'),
                layers.MaxPool2D((2,2)),

                layers.Conv2D(128, (3,3), activation = 'relu'),
                layers.MaxPool2D((2,2)),

                layers.Conv2D(64, (3,3) , kernel_regularizer=tf.keras.regularizers.l2(0.01),),
                layers.BatchNormalization(),
                layers.Activation('relu'),

                layers.MaxPool2D((2,2)),
                layers.Conv2D(64, (3,3), activation = 'relu'),
                layers.MaxPool2D((2,2)),

                layers.Conv2D(64, (3,3) , kernel_regularizer=tf.keras.regularizers.l2(0.01),),
                layers.BatchNormalization(),
                layers.Activation('relu'),
                layers.MaxPool2D((2,2)),

                layers.Conv2D(32, (3,3), activation = 'relu'),
                layers.MaxPool2D((2,2)),

                layers.Flatten(),
                layers.Dense(32, activation = 'relu'),
                layers.Dense(n_classes, activation = 'softmax')
                ])
        # Define the optimizer :
        optimizer = RMSprop(learning_rate=learning_rate, rho=rho, epsilon=epsilon)
        # Compile the model:
        model.compile(
            optimizer=optimizer, 
            loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
            metrics=["accuracy"])

        
        # Print the model summary
        model.summary()
        return model

    def build_model(self):
        """
        Build the model by adding custom layers and compile it.
        Save the updated model to the specified path.
        """

        # Prepare the full model with custom layers and specified parameters
        self.full_model = self.prepare_model(
            input_shape = self.config.params_image_size, 
            n_classes = self.config.params_n_classes, 
            learning_rate = self.config.params_learning_rate, 
            rho = self.config.params_rho, 
            epsilon = self.config.params_epsilon,
        )

        # Save the updated full model to the specified path
        self.save_model(path=self.config.built_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):        
        model.save(path)

In [8]:
try:
    # Initialize the ConfigurationManager to load configuration and parameter settings
    config = ConfigurationManager()
    
    # Retrieve the configuration settings for preparing the base model
    prepare_model_config = config.get_prepare_model_config()
    
    # Initialize the PrepareModel process with the configuration settings
    prepare_model = PrepareModel(config=prepare_model_config)
    
    # Build the model by adding custom layers and compiling it with specified parameters
    prepare_model.build_model()
    
except Exception as e:
    # Log the exception details and raise it
    logger.error(f"An error occurred: {e}")
    raise e

[2024-08-03 23:40:34,685: INFO: common: YAML file: configYaml\config.yaml loaded successfully]
[2024-08-03 23:40:34,696: INFO: common: YAML file: configYaml\params.yaml loaded successfully]
[2024-08-03 23:40:34,698: INFO: common: Created directory at: artifacts]
[2024-08-03 23:40:34,700: INFO: common: Created directory at: artifacts/prepare_built_model]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 128)     3584      
                                                                 
 batch_normalization (BatchN  (None, 254, 254, 128)    512       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 254, 254, 128)     0         
                                                                 
 max_pooling2d (MaxPooling2D  